
Identify the Goal, Metrics(KPI), Audience,

1. Problem Statement
1. Stakeholders

Parameters, Descriptions, Criteria

What visualizations do I need? How should information be presented?


Data: https://www.kaggle.com/datasets/jmullan/agricultural-land-values-19972017?select=Combined+Data+-+Pasture.csv

See if you can find the same granularity data for crop yields so that more comparison can be made and a yield/value metric can be extracted.

Problem Statement:
Does higher land value mean higher yield?  What are the trends for the two key metrics over time? Which regions, landuses, and states have the highest values, yields and values/yields. Show land value and crop yield at starting year, middle year and last available year on a map visual.

Agri Value by year
Agri Yield by year


^ Is this scope creep? It doesn't feel like just using the value would be enough for a compelling SQL DB structure, PBI dashboard or data story compared to the interplay between the value of the land, yield, and type of crop.


1. Set up SQL DB **
1. Load unclean land value data and use Python to clean data **
1. Load unclean crop production data and use Python to clean data **
1. Populate SQL DB with 2 tables, one for landval and one for cropprod **
1. Join tables into view and use to pull into Dashboard
1. Use data to create visuals that visualize the relationship between the data and answer the questions in the problem statements.

## SQL DB Creation

In [ ]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
%%sql
sqlite:///agri_landval.db

In [ ]:
%%sql
create table agricultural_landvalue(State varchar, Region varchar, [Land Use] varchar, Year int, Acre_Value float);



 * sqlite:///agri_landval.db
   sqlite:///test.db
Done.


[]

## Land Value

In [ ]:
# Load Data into SQL
import sqlite3
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
conn = sqlite3.connect('agri_landval.db')




In [ ]:
cropland_df = pd.read_csv('Combined Data - Cropland.csv', header = 4)
cropland_df.drop(columns = ['2','h','2016-2017'],index=[0,1],inplace=True)
cropland_df.replace('[1234567890]/', '', regex=True,inplace=True)
cropland_df.dropna(inplace=True)
cropland_df.reset_index(drop=True,inplace=True)


farmre_df = pd.read_csv('Combined Data - Farm Real Estate.csv',header=4,encoding_errors = 'replace')
farmre_df.drop(columns = ['1','h','2016-2017'],index=[0,1],inplace=True)
farmre_df.replace('[1234567890]/', '', regex=True,inplace=True)
farmre_df.dropna(inplace=True)
farmre_df.reset_index(drop=True,inplace=True)


pasture_df = pd.read_csv('Combined Data - Pasture.csv',header=4)
pasture_df.drop(columns = ['5','h','2016-2017'],index=[0,1],inplace=True)
pasture_df.replace('[1234567890]/', '', regex=True,inplace=True)
pasture_df.dropna(inplace=True)
pasture_df.reset_index(drop=True,inplace=True)

REGIONS= ("Northeast", "Lake", "Corn Belt", "Northern Plains", "Appalachian", "Southeast", "Delta", "Southern Plains", "Mountain", "Pacific")


In [ ]:


cropland_reg = cropland_df[cropland_df['Region and state'].isin(REGIONS)]
farmre_reg = farmre_df[farmre_df['Region and state'].isin(REGIONS)]
pasture_reg = pasture_df[pasture_df['Region and State'].isin(REGIONS)]


# cropland_dict = {}
# farmre_dict = {}
# pasture_dict = {}


# for i in REGIONS:
#   cropland_dict[i] = cropland_reg[cropland_reg['Region and state'] == i].index.values[0]
#   farmre_dict[i] = farmre_reg[farmre_reg['Region and state'] == i].index.values[0]
#   pasture_dict[i] = pasture_reg[pasture_reg['Region and State'] == i].index.values[0]

# cropland_list = list(cropland_dict.values())
# farmre_list = list(farmre_dict.values())
# pasture_list = list(pasture_dict.values())

cropland_list = list(cropland_reg.index)
farmre_list = list(farmre_reg.index)
pasture_list = list(pasture_reg.index)


cropland_list.append(cropland_df.shape[0])
farmre_list.append(farmre_df.shape[0])
pasture_list.append(pasture_df.shape[0])

cropland_idx_dict = {}
farmre_idx_dict = {}
pasture_idx_dict = {}

for i in range(0,len(cropland_list)-1):
  cropland_idx_dict[REGIONS[i]] = [cropland_list[i]+1,cropland_list[i+1]]

for i in range(0,len(farmre_list)-1):
  farmre_idx_dict[REGIONS[i]] = [farmre_list[i]+1,farmre_list[i+1]]

for i in range(0,len(pasture_list)-1):
  pasture_idx_dict[REGIONS[i]] = [pasture_list[i]+1,pasture_list[i+1]]

cropland_df.rename(columns={'Region and state':'State'},inplace=True)
cropland_df['Region'] = ''
for i in cropland_idx_dict.keys():
  cropland_df.loc[cropland_idx_dict[i][0]:cropland_idx_dict[i][1], 'Region'] = i
cropland_df.drop(index=cropland_list[:-1],inplace=True)
cropland_df.drop(index=[53],inplace=True)
cropland_df.reset_index(drop=True,inplace=True)
cropland = pd.melt(cropland_df,id_vars=['State','Region'],var_name='Year',value_name='Acre Value')
cropland['Acre Value'] = pd.to_numeric(cropland['Acre Value'],errors='coerce')
cropland.dropna(inplace=True)
cropland['Land Use'] = 'Cropland'
cropland.reset_index(drop=True,inplace=True)

farmre_df.rename(columns={'Region and state':'State'},inplace=True)
farmre_df['Region'] = ''
for i in farmre_idx_dict.keys():
  farmre_df.loc[farmre_idx_dict[i][0]:farmre_idx_dict[i][1], 'Region'] = i

farmre_df.drop(index=farmre_list[:-1],inplace=True)
farmre_df.drop(index=[58],inplace=True)
farmre_df.reset_index(drop=True,inplace=True)
farmre = pd.melt(farmre_df,id_vars=['State','Region'],var_name='Year',value_name='Acre Value')
farmre['Acre Value'] = pd.to_numeric(farmre['Acre Value'],errors='coerce')
farmre.dropna(inplace=True)
farmre['Land Use'] = 'Farm Real Estate'
farmre.reset_index(drop=True,inplace=True)

pasture_df.rename(columns={'Region and State':'State'},inplace=True)
pasture_df['Region'] = ''
for i in pasture_idx_dict.keys():
  pasture_df.loc[pasture_idx_dict[i][0]:pasture_idx_dict[i][1], 'Region'] = i

pasture_df.drop(index=pasture_list[:-1],inplace=True)
pasture_df.drop(index=[52],inplace=True)
pasture_df.reset_index(drop=True,inplace=True)
pasture = pd.melt(pasture_df,id_vars=['State','Region'],var_name='Year',value_name='Acre Value')
pasture['Acre Value'] = pd.to_numeric(pasture['Acre Value'],errors='coerce')
pasture.dropna(inplace=True)
pasture['Land Use'] = 'Pasture'
pasture.reset_index(drop=True,inplace=True)

df = pd.concat([cropland,farmre,pasture],ignore_index=True)


In [ ]:
df.to_sql('agricultural_landvalue',conn,if_exists='replace',index=False)

2765

In [ ]:
%%sql
select * from agricultural_landvalue LIMIT 100;


 * sqlite:///agri_landval.db
   sqlite:///test.db
Done.


State,Region,Year,Acre Value,Land Use
Delaware,Northeast,1997,2500.0,Cropland
Maryland,Northeast,1997,3050.0,Cropland
New Jersey,Northeast,1997,8500.0,Cropland
New York,Northeast,1997,1020.0,Cropland
Pennsylvania,Northeast,1997,2650.0,Cropland
Other States,Northeast,1997,4260.0,Cropland
Michigan,Lake,1997,1360.0,Cropland
Minnesota,Lake,1997,1090.0,Cropland
Wisconsin,Lake,1997,1040.0,Cropland
Illinois,Corn Belt,1997,2070.0,Cropland


## Crop Production

In [ ]:
cropprod_df = pd.read_csv('Crop Production in $.csv')
cropprod_df.head()
cropprod_df = cropprod_df[['Year','State','Value']]
cropprod_df['Value'] = cropprod_df['Value'].str.replace(',','')


cropprod_df['Value'] = pd.to_numeric(cropprod_df['Value'], errors='coerce')
cropprod_df.dropna(inplace=True)
cropprod_df['Value'] = cropprod_df['Value'].astype(int)
cropprod_df.reset_index(drop=True,inplace=True)
cropprod_df.rename(columns={'Value':'Crop Yield'},inplace=True)
cropprod_df.head()
cropprod_df.to_sql('crop_production',conn,if_exists='replace',index=False)

528

In [ ]:
%%sql
SELECT * FROM crop_production LIMIT 100;

 * sqlite:///agri_landval.db
   sqlite:///test.db
Done.


Year,State,Crop Yield
2017,ALABAMA,1043656000
2017,ARIZONA,2576661000
2017,ARKANSAS,3819332000
2017,CALIFORNIA,33466945000
2017,CONNECTICUT,38583000
2017,FLORIDA,3860095000
2017,GEORGIA,2988429000
2017,HAWAII,117220000
2017,IDAHO,3028601000
2017,ILLINOIS,13891611000
